In [2]:
import subprocess
from itertools import product
from multiprocessing import Pool, Queue
import time
import argparse
import threading

def parse_arguments():
    parser = argparse.ArgumentParser(description="Task Executor for GPU-based Evaluation")
    parser.add_argument("--bench_type", type=str, required=True, help="Benchmark type (e.g., MMLU or TrustLLM)")
    parser.add_argument("--model_type", type=str, required=True, help="Model type (e.g., Kangaroo or Baseline)")
    parser.add_argument("--GPU_number", type=int, required=True, help="Number of GPUs to use")
    parser.add_argument("--interval", type=int, default=20, help="Interval between command executions in seconds")
    return parser.parse_args()

def get_free_gpus():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'], stdout=subprocess.PIPE)
    gpu_memory = result.stdout.decode('utf-8').strip().split('\n')
    gpu_memory = [(i, int(mem)) for i, mem in enumerate(gpu_memory)]
    sorted_gpus = sorted(gpu_memory, key=lambda x: x[1], reverse=True)
    return [gpu[0] for gpu in sorted_gpus if gpu[1] > 16000]


In [3]:
get_free_gpus()

[3, 5, 7]